# Geomechanical Injection Scenario Toolkit (GIST)

#Disclaimer
GIST aims to give the _gist_ of a wide range of potential scenarios and aid collective decision making when responding to seismicity.

The results of GIST are entirely dependent upon the inputs provided, which may be incomplete or inaccurate.

There are other potentially plausible inducement scenarios that are not considered, including fluid migration into the basement, 
out-of-zone poroelastic stressing, or hydraulic fracturing.

None of the individual models produced by GIST accurately represent what happens in the subsurface and cannot be credibly used 
to accurately assign liability or responsibility for seismicity.

"All models are wrong, but some are useful" - George Box, 1976

## Prerequisites

Assumes InjectionSQLScheduled completed successfully and injection data are sampled uniformly in time

##Install Dependencies
- geopandas
- gistMC.py
- eqSQL.py
- gistPlots.py
- numpy
- scipy
- pandas


In [0]:
%restart_python

In [0]:
%run "/Workspace/_utils/Utility_Functions"

In [0]:
!pip install geopandas
!pip install geodatasets
!pip install contextily
#! pip install folium matplotlib mapclassify contextily

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 119.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 139.0 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 4.9 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

##Paths

In [0]:
# Paths
homePath='/Workspace/Users/bill.curry@exxonmobil.com/'
# Injection data path 
injPath=homePath+'injection/WeeklyRun/ScheduledOutput/'
# GIST library path
gistPath=homePath+'GIST/'

##Libraries

- numpy
- scipy
- pandas
- matplotlib
- geopandas
- pyspark


In [0]:
import sys
sys.path.append(gistPath+'lib')

In [0]:
#Databricks-specific
#import dataBricksConfig as db
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("eqSQL").getOrCreate()

In [0]:
import numpy as np
import pandas as pd
import os
import gistMC as gi
import eqSQL as es
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
#import contextily as cx

In [0]:
import gistPlots as gp

#1. Select Event

In [0]:
eventID='texnet2024oqfb'

In [0]:
runPath=gistPath+'/runs/'+eventID+'/'

In [0]:
EQ=pd.read_csv(runPath+'EQ.csv').loc[0]
print(EQ)

Unnamed: 0                                                0
SeismicEventId                                      2481836
DataSource                        TexNet Earthquake Catalog
DataSourceUrl                                           NaN
EventID                                      texnet2024oqfb
EventTimeUtc                            2024-07-27 02:11:07
EventTimeInLocalTimeZone               2024-07-26T21:11:07Z
EventTimeZone                                           CST
EventType                                        Earthquake
DepthKm                                            8.837891
DepthErrorKm                                       0.636678
Magnitude                                           1.64016
MagnitudeError                                          NaN
MagnitudeType                                    ml(texnet)
Location                                      Western Texas
Status                                                final
Latitude                                

#2. Initial Run

##2.1 Subsurface Model Parameters

###Inputs:

In [0]:
# Binary deep/shallow parameter
deepOrShallow='Deep'
# Depth from surface (ft)
# Distinguishes deep/shallow where we don't have a horizon 
depthCutoff=8000.

In [0]:
nRealizations=500

In [0]:
# Water density minimum/maximum (kg/m3)
WaterDensityMin=1015.
WaterDensityMax=1025.
# Water viscosity minimum/maximum (Pa.s)
WaterViscosityMin=0.000799
WaterViscosityMax=0.000801
# Fluid Compressibility minimum/maximum (1/Pa)
FluidCompressibilityMin=0.000000000359
FluidCompressibilityMax=0.000000000361

In [0]:
# Porosity in percent
PorosityPercentMin=3.
PorosityPercentMax=15.

# Permeability in millidarcies
PermMDMin=5.
PermMDMax=500.

# Thickness in feet
ThicknessFTMin=200.
ThicknessFTMax=2000.

# Vertical compressibility minimum/maximum (1/Pa)
VerticalCompressibilityMin=0.00000000107
VerticalCompressibilityMax=0.00000000109

In [0]:
# Verbosity - 0=silent, 1=some, 2=lots
verb=0
# Minimum Pressure change to care about in PSI
dPCutoff=0.5
# Maximum Number of wells to plot
nWells=50
# How far back in time to plot in years
minYear=-40

### Set Interval-Specific Paths...


In [0]:
# Set an output directory for this earthquake and this interval
runIntervalPath=runPath+deepOrShallow+'/'
initialRunIntervalPath=runIntervalPath+'initialRun/'
updatedRunIntervalPath=runIntervalPath+'udpatedRun/'
forecastRunIntervalPath=runIntervalPath+'forecastRun/'
disposalPath=runIntervalPath+'updatedDisposal/'
################################################
# Output prefix for realizations of parameters #
################################################
RealizationPrefix=runIntervalPath+'MC'

In [0]:
# Point to appropriate well and injection files from initial database
if deepOrShallow=='Deep':
  WellFile=injPath+'/deep.csv'
  InjFile=injPath+'/deepReg.csv'
elif deepOrShallow=='Shallow':
  WellFile=injPath+'/shallow.csv'
  InjFile=injPath+'/shallowReg.csv'

##2.2 Results

###Compute

###Output

#3. Correct Data

##3.1 Export Disposal Data

##3.2 Import Updated Disposal Data

In [0]:

udpatedSelectedWellsDF=pd.read_csv(disposalPath+'selectedWells.csv')
updatedIgnoredWellsDF=pd.read_csv(disposalPath+'ignoredWells.csv')
updatedWellsFile=disposalPath+'allInZoneWells.csv'
updatedInjFile=disposalPath+'inj.csv'

In [0]:
# I need to merge the updated disposal information with the prior stuff
# Load updated injection file
# Well comparisons
# Get list of well IDs in the updated injection file
# Get list of well IDs in updated well file
# Check to see overlap in well IDs
#    updated vs. prior selected wells
#    updated vs. prior ignored wells
# FilteredIgnored =  prior ignored wells - (wells in updated file and prior ignored wells)
# Merge FilteredIgnored with with updated wells
# Load injection
#    Check for time sampling of injection - all wells must be the same
#    Get overall time vector - convert to days
#    Loop over wells in well file
#      Get all disposal data for well ID
#      Convert date values to days
#      
#mergedInjFile=

#4. Updated Analysis

##4.1 Refined Parameters

In [0]:
UpdatednRealizations=2000

In [0]:
# Porosity in percent
UpdatedPorosityPercentMin=3.
UpdatedPorosityPercentMax=15.

# Permeability in millidarcies
UpdatedPermMDMin=10.
UpdatedPermMDMax=500.

# Thickness in feet
UpdatedThicknessFTMin=500.
UpdatedThicknessFTMax=1200.

# Vertical compressibility minimum/maximum (1/Pa)
UpdatedVerticalCompressibilityMin=0.00000000107
UpdatedVerticalCompressibilityMax=0.00000000109

In [0]:
# Verbosity - 0=silent, 1=some, 2=lots
verb=0
# Minimum Pressure change to care about in PSI
dPCutoff=0.5
# Maximum Number of wells to plot
nWells=50
# How far back in time to plot in years
minYear=-40

In [0]:
gist2=gi.gistMC(nReal=UpdatednRealizations)
gist2.initPP(rho0_min=WaterDensityMin,
             rho0_max=WaterDensityMax,
             phi_min=UpdatedPorosityPercentMin,
             phi_max=UpdatedPorosityPercentMax,
             kMD_min=UpdatedPermMDMin,
             kMD_max=UpdatedPermMDMax,
             h_min=UpdatedThicknessFTMin,
             h_max=UpdatedThicknessFTMax,
             alphav_min=VerticalCompressibilityMin,
             alphav_max=VerticalCompressibilityMax,
             beta_min=FluidCompressibilityMin,
             beta_max=FluidCompressibilityMax)

##4.2 Updated Disposal Rates

In [0]:
gist2.addWells(userWellFile=updatedWellsFile,userInjFile=updatedInjFile,verbose=verb)

In [0]:
updatedSelectedWellsDF,updatedIgnoredWellsDF,updatedInjDF=gist2.findWells(EQ,PE=False,verbose=2)

 gistMC.findWells: iw= 0  of  2604  start date is  1931-12-22
 gistMC.findWells injectionDays 33821
 gistMC.findWells: iw= 1  of  2604  start date is  1938-07-18
 gistMC.findWells injectionDays 31421
 gistMC.findWells: iw= 2  of  2604  start date is  1931-12-22
 gistMC.findWells injectionDays 33821
 gistMC.findWells: iw= 3  of  2604  start date is  1931-12-22
 gistMC.findWells injectionDays 33821
 gistMC.findWells: iw= 4  of  2604  start date is  1994-10-01
 gistMC.findWells injectionDays 10892
 gistMC.findWells: iw= 5  of  2604  start date is  1996-01-01
 gistMC.findWells injectionDays 10435
 gistMC.findWells: iw= 6  of  2604  start date is  1996-08-28
 gistMC.findWells injectionDays 10195
 gistMC.findWells: iw= 7  of  2604  start date is  1995-07-01
 gistMC.findWells injectionDays 10619
 gistMC.findWells: iw= 8  of  2604  start date is  2000-03-01
 gistMC.findWells injectionDays 8914
 gistMC.findWells: iw= 9  of  2604  start date is  1996-09-24
 gistMC.findWells injectionDays 10168
 

##4.3 Updated Run

In [0]:
updatedScenarioDF=gist2.runPressureScenarios(EQ,updatedSelectedWellsDF,updatedInjDF,verbose=verb)

In [0]:
updatedFilteredDF,updatedOrderedWellList=gi.summarizePPResults(updatedScenarioDF,updatedSelectedWellsDF,threshold=dPCutoff,nOrder=nWells,verbose=verb)
updatedDisaggregationDF=gi.prepDisaggregationPlot(updatedFilteredDF,updatedOrderedWellList,jitter=0.1,verbose=0)
updatedDiffRange=(min(gist2.diffPPVec),max(gist2.diffPPVec))
updatedrtDF,updatedMergedWellsDF = gi.prepRTPlot(updatedSelectedWellsDF,updatedIgnoredWellsDF,minYear,updatedDiffRange,clipYear=False)
updatedWinWellsDF,updatedWinInjDF=gi.getWinWells(updatedFilteredDF,updatedSelectedWellsDF,updatedInjDF)

In [0]:
updatedScenarioTSRDF,updatedDPTimeSeriesR,updatedWellIDsR,updatedDayVecR = gist2.runPressureScenariosTimeSeries(EQ,updatedWinWellsDF,updatedWinInjDF,verbose=verb)

In [0]:
updatedTotalPPQuantilesDF=gi.prepTotalPressureTimeSeriesPlot(updatedDPTimeSeriesR,updatedDayVecR,nQuantiles=11,epoch=pd.to_datetime('1970-01-01'),verbose=1)
updatedTotalPPSpaghettiDF=gi.prepTotalPressureTimeSeriesSpaghettiPlot(updatedDPTimeSeriesR,updatedDayVecR,gist2.diffPPVec,epoch=pd.to_datetime('1970-01-01'),verbose=1)
updatedAllPPQuantilesDF,updatedAllPPSpaghettiDF=gi.getPerWellPressureTimeSeriesSpaghettiAndQuantiles(updatedDPTimeSeriesR,updatedDayVecR,gist2.diffPPVec,updatedWellIDsR,nQuantiles=11,epoch=pd.to_datetime('01-01-1970'))
updatedWellPressureDict=gi.prepPressureAndDisposalTimeSeriesPlots(updatedAllPPQuantilesDF,updatedAllPPSpaghettiDF,updatedWinWellsDF,updatedWinInjDF,updatedOrderedWellList[:-1],verbose=0)

prepTotalPressureTimeSeriesPlot: deltaPP.shape= (18, 2000, 1415)  dayVec.shape= (1415,)
prepTotalPressureTimeSeriesPlot: totalDeltaPP.shape= (2000, 1415)
prepTotalPressureTimeSeriesPlot: quantiles: [0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]
prepTotalPressureTimeSeriesSpaghettiPlot: deltaPP.shape= (18, 2000, 1415)  dayVec.shape= (1415,)
prepTotalPressureTimeSeriesSpaghettiPlot: totalDeltaPP.shape= (2000, 1415)


In [0]:
updatedSensitivityDF,updatedSensitivitySumDF = gist2.getPressureSensitivity(updatedWinInjDF,updatedWinWellsDF,EQ)

##4.4 Updated Run Output

In [0]:
gist2.writeRealizations(updatedRunIntervalPath+'PorePressureRealizations.csv')
updatedScenarioDF.to_csv(updatedRunIntervalPath+'scenarios.csv')
updatedFilteredDF.to_csv(updatedRunIntervalPath+'filteredScenarios.csv')
pd.Series(data=updatedOrderedWellList).to_csv(updatedRunIntervalPath+'wellOrder.csv')
updatedMergedWellsDF.to_csv(updatedRunIntervalPath+'RTwells.csv')
updatedrtDF.to_csv(updatedRunIntervalPath+'RTDF.csv')
updatedDisaggregationDF.to_csv(updatedRunIntervalPath+'disaggregation.csv')
updatedTotalPPQuantilesDF.to_csv(updatedRunIntervalPath+'totalPPQuantiles.csv')
updatedTotalPPSpaghettiDF.to_csv(updatedRunIntervalPath+'totalPPSpaghetti.csv')

In [0]:
i=0
for wellDictKey, wellDictValue in updatedWellPressureDict.items():
  wellID=wellDictValue['WellInfo']['ID'].to_list()[0]
  wellFilePrefix='/perWell/well_'+str(i)+'_'
  wellDictValue['PPQuantiles'].to_csv(updatedRunIntervalPath+wellFilePrefix+'PPQuantiles.csv')
  wellDictValue['Disposal'].to_csv(updatedRunIntervalPath+wellFilePrefix+'Disposal.csv')
  wellDictValue['WellInfo'].to_csv(updatedRunIntervalPath+wellFilePrefix+'WellInfo.csv')
  wellDictValue['Spaghetti'].to_csv(updatedRunIntervalPath+wellFilePrefix+'Spaghetti.csv')
  print('well',i,', ID:',wellID,' completed')
  i=i+1

well 0 , ID: 2121094  completed
well 1 , ID: 2121271  completed
well 2 , ID: 2123938  completed
well 3 , ID: 2117885  completed
well 4 , ID: 2111946  completed
well 5 , ID: 2109094  completed
well 6 , ID: 2116501  completed
well 7 , ID: 2112288  completed
well 8 , ID: 2081404  completed
well 9 , ID: 2114817  completed
well 10 , ID: 2112104  completed
well 11 , ID: 2111358  completed
well 12 , ID: 2056720  completed
well 13 , ID: 2121191  completed
well 14 , ID: 2113898  completed
well 15 , ID: 2111690  completed
well 16 , ID: 2118993  completed
well 17 , ID: 2121195  completed


In [0]:
updatedScenarioTSRDF.to_csv(updatedRunIntervalPath+'materialScenariosR.csv')
np.savez_compressed(updatedRunIntervalPath+'timeSeriesR.npz', deltaPP=updatedDPTimeSeriesR,dayVec=updatedDayVecR,wellIDs=updatedWellIDsR)
updatedSensitivityDF.to_csv(updatedRunIntervalPath+'sensitivity.csv')
updatedSensitivitySumDF.to_csv(updatedRunIntervalPath+'sensitivitySum.csv')
#rateDF.to_csv(updatedRunIntervalPath+'rates.csv')

In [0]:
# Write out gist object to disk
import pickle
with open(updatedRunIntervalPath+'gist.pkl', 'wb') as file:
  pickle.dump(gist2, file)